# Differential Geometry #
learning from the book *An Introduction to Riemannian Geometry* by Leonor Godinho and José Natário.

## topological manifold ##
a topological manifold $M$ of dimension $n$ is a topological space with the following properties

Primary
- Each point $p \in M$ has a neborhood homeomorphic to an open subset of $\mathbb{R}^n$

Nessisary to maintain sanity
- $M$ is **Hausdorff** space
- $M$ is a **second-countable** space

**second-countable** space: is a topological space whose topology has a countable base

**Hausdorff** space: is a topological space where for any two distinct points there exist neighbourhoods of each which are disjoint from each other.


In [1]:
import Test.QuickCheck

In [ ]:
class LinearSpace a where -- is a Monoid with the addition of scalarMult
    vectorAdd :: a -> a -> a
    scalarMult :: Double -> a -> a
    vectorZero :: a

class MetricSpace a where
    distanceSquared :: a -> a -> Double
    findBoundingBox :: [a] -> (a, Double)

class Domain a where
    inDomain :: (LinearSpace s, MetricSpace s) =>  a s -> s -> Bool

class Homeomorphism a where
    move :: a n m -> a n m -> n -> n

In [ ]:
--data Point = R2 Double Double | R3 Double Double Double deriving Show -- | Rn Integer [Double] 
data R2 = R2 Double Double deriving Show
data R3 = R3 Double Double Double deriving Show

type Curve a = [a]

data Region a = Square a Double | Disk a Double | Annulus a Double Double deriving (Eq, Show)

data Chart a b = Chart {domain::Region a, parameterization::a -> b, coordinateSystem::b -> a}

data ChartCurve a b = ChartCurve (Curve a) (Chart a b)

type Atlas a b = [Chart a b]

## Graphing basics ##

In [ ]:
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Grid
import Graphics.Rendering.Chart.Backend.Cairo

linspace :: Double -> Double -> Int -> [Double]
linspace start stop num =
    let dx = (stop - start)/fromIntegral (num - 1)
    in [fromIntegral i*dx + start| i <- [0 .. (num - 1)]]

lerp :: LinearSpace a => Double -> a -> a -> a
lerp t p_0 p_1 = vectorAdd (scalarMult (1-t) p_0) (scalarMult t p_1)

lerp' :: LinearSpace a => a -> a -> a
lerp' p_0 p_1 = vectorAdd p_1 (scalarMult (-1) p_0)

bezierRedux :: (LinearSpace a) => Double -> [a] -> a
bezierRedux _ [] = error "One or more points must be given"
bezierRedux _ [point] = point
bezierRedux t points = bezierRedux t [ lerp t p1 p2 | (p1, p2) <- zip (init points) (tail points)]

bezierRedux' :: (LinearSpace a) => Double -> [a] -> a
--bezierRedux' _ [] = error "One or more points must be given"
bezierRedux' _ [point] = vectorZero
bezierRedux' t points = vectorAdd (bezierRedux' t [ lerp t p1 p2 | (p1, p2) <- zip (init points) (tail points)]) (bezierRedux t [ lerp' p1 p2 | (p1, p2) <- zip (init points) (tail points)])

bezierCurveClosed :: (LinearSpace a) => Int -> [a] -> Curve a
bezierCurveClosed resolution points = [bezierRedux t (last points:points) | t <- linspace 0 1 resolution]

bezierCurve :: (LinearSpace a) => Int -> [a] -> Curve a
bezierCurve resolution points = [bezierRedux t points | t <- linspace 0 1 resolution]

--derivative of the curve
bezierCurve' :: (LinearSpace a) => Int -> [a] -> Curve a
bezierCurve' resolution points = [bezierRedux' t points | t <- linspace 0 1 resolution]

curveFilterBoundary :: (Domain a, LinearSpace b, MetricSpace b) => a b -> Curve b -> ([Curve b], [Curve b])
curveFilterBoundary region curve = (filter (inDomain region . head) curves, filter ((not . inDomain region) . head) curves)
    where curves = curveSplitOnBoundary region curve

curveSplitOnBoundary :: (Domain a, LinearSpace b, MetricSpace b) => a b -> Curve b -> [Curve b]
curveSplitOnBoundary region [] = []
curveSplitOnBoundary region curve = c:curveSplitOnBoundary region cs
    where
    c = curveTakeFirstBoundary region curve
    cs = drop (length c) curve

curveTakeFirstBoundary :: (Domain a, LinearSpace b, MetricSpace b) => a b -> Curve b -> Curve b
curveTakeFirstBoundary _ [point] = [point]
curveTakeFirstBoundary region (p1:p2:other) | inDomain region p1 == inDomain region p2 = p1:curveTakeFirstBoundary region (p2:other)
                                            | otherwise = [p1]

binding :: (Monad m) => [m a] -> m a
binding [a] = a
binding (a:as) = a >> binding as

plotSideBySide plot1 plot2 = aboveN [ besideN [ layoutToGrid (execEC plot1), layoutToGrid (execEC plot2)]]
plotNSideBySide plots = aboveN [ besideN [ layoutToGrid (execEC plot) | plot <- plots]]
plotGrid plots = aboveN [ besideN [ layoutToGrid (execEC plot) | plot <- rowOfPlots] | rowOfPlots <- plots]

plotR2Points pointlist = plot (points "point in R2" [(x, y) | (R2 x y) <- pointlist])
plotR2Line pointlist = plot (line "line in R2" [[(x, y) | (R2 x y) <- pointlist]])
plotR2Lines lineList = plot (line "lines in R2" [[(x, y) | (R2 x y) <- pointList] | pointList <- lineList])

plotCurveTransformed func curve = plotCurve [func point | point <- curve]
plotCurvesTransformed func curves = plotCurves [[func point | point <- curve] | curve <- curves]

plotCurve = plotR2Line 
plotCurves = plotR2Lines

--plotTaylorSeries functions x = toRenderable $ binding [plot (line name [eval function x]) | (function, name) <- functions]

## R2 and R3 ##

In [ ]:
(~=) :: (Num a, Ord a, Fractional a) => a -> a -> Bool
(~=) a b = abs(a - b) < 1e-5

instance MetricSpace R2 where
    distanceSquared (R2 x1 y1) (R2 x2 y2) = (x1 - x2)^2 + (y1 - y2)^2
    findBoundingBox points = (R2 cx cy, maximum [rx, ry]) where
        (xs, ys) = ([x | (R2 x _) <- points], [y | (R2 _ y) <- points])
        (xmax, xmin) = (maximum xs, minimum xs)
        (ymax, ymin) = (maximum ys, minimum ys)
        (cx, cy) = ((xmax + xmin)/2, (ymax + ymin)/2)
        (rx, ry) = ((xmax - xmin)/2, (ymax - ymin)/2)
    
instance MetricSpace R3 where
    distanceSquared (R3 x1 y1 z1) (R3 x2 y2 z2) = (x1 - x2)^2 + (y1 - y2)^2 + (z1 - z2)^2
    findBoundingBox points = (R3 cx cy cz, maximum [rx, ry, rz]) where
        (xs, ys, zs) = ([x | (R3 x _ _) <- points], [y | (R3 _ y _) <- points], [z | (R3 _ _ z) <- points])
        (xmax, xmin) = (maximum xs, minimum xs)
        (ymax, ymin) = (maximum ys, minimum ys)
        (zmax, zmin) = (maximum zs, minimum zs)
        (cx, cy, cz) = ((xmax + xmin)/2, (ymax + ymin)/2, (zmax + zmin)/2)
        (rx, ry, rz) = ((xmax - xmin)/2, (ymax - ymin)/2, (zmax - zmin)/2)


instance LinearSpace Double where
    vectorAdd a b = a + b
    scalarMult s a = s*a
    vectorZero = 0.0

instance LinearSpace R2 where
    vectorAdd (R2 x1 y1) (R2 x2 y2) = R2 (x1 + x2) (y1 + y2)
    scalarMult s (R2 x y) = R2 (s*x) (s*y)
    vectorZero = R2 0 0
    
instance LinearSpace R3 where
    vectorAdd (R3 x1 y1 z1) (R3 x2 y2 z2) = R3 (x1 + x2) (y1 + y2) (z1 + z2)
    scalarMult s (R3 x y z) = R3 (s*x) (s*y) (s*z)
    vectorZero = R3 0 0 0

instance Eq R2 where
    (==) (R2 a b) (R2 c d) = (a ~= c) && (b ~= d)

instance Eq R3 where
    (==) (R3 a b c) (R3 d f g) = (a ~= d) && (b ~= f) && (c ~= g)

-- for generating arbitrary elements of R2 and R3

instance Arbitrary R2 where
    arbitrary = do
        a <- arbitrary
--        b <- arbitrary
--        return (R2 (a/10) (b/10))
        R2 a <$> arbitrary

instance Arbitrary R3 where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        c <- arbitrary
        return (R3 (a/10) (b/10) (c/10))

genR2 :: Gen R2
genR2 = arbitrary

genR2' :: Gen R2
genR2' = do
        a <- arbitrary
--        b <- arbitrary
        R2 a <$> arbitrary

genR3 :: Gen R3
genR3 = arbitrary

## Region ##
Regions are defined on R2 or R3

In [ ]:
instance Domain Region where
    inDomain (Square center halfWidth) point = 2*boxHalfWidth < halfWidth
        where (_, boxHalfWidth) = findBoundingBox [center, point]
    inDomain (Disk center r) point = distanceSquared center point < r^2
    inDomain (Annulus center r1 r2) point = (distanceSquared center point > r1^2) && (distanceSquared center point < r2^2)

_boundaryRes = 200

boundary :: Region R2 -> [[R2]]
boundary (Square (R2 x y) hd) = [concat [[lerp t corner1 corner2 | t <- linspace 0 1 _boundaryRes] | (corner1, corner2) <- zip (init corners) (tail corners)]]
    where corners = [R2 (x - hd) (y - hd), R2 (x - hd) (y + hd), R2 (x + hd) (y + hd), R2 (x + hd) (y - hd), R2 (x - hd) (y - hd)]
boundary (Disk (R2 x y) r) = [[R2 (x + r*sin t) (y + r*cos t) | t <- linspace 0 (2*pi) (_boundaryRes*4)]]
boundary (Annulus (R2 x y) r1 r2) = [[R2 (x + r1*sin t) (y + r1*cos t) | t <- linspace 0 (2*pi) (_boundaryRes*2)], [R2 (x + r2*sin t) (y + r2*cos t) | t <- linspace 0 (2*pi) (_boundaryRes*2)]]

-- for generating arbitrary regions

genPos :: Gen Double
genPos = abs `fmap` (arbitrary :: Gen Double) `suchThat` (> 0)

arbitrarySquareRegion = do
    point <- arbitrary
    Square point <$> genPos
    
arbitraryDiskRegion = do
    point <- arbitrary
    Disk point <$> genPos

ord (a, b) | b < a = (b, a)
           | otherwise = (a, b)

arbitraryAnnulusRegion = do
    point <- arbitrary
    a <- genPos
    b <- genPos
    let (r1, r2) = ord (a, b)
    return (Annulus point r1 r2)

instance (Arbitrary a) => Arbitrary (Region a) where
    arbitrary = oneof [arbitrarySquareRegion, arbitraryDiskRegion, arbitraryAnnulusRegion]
    
genRegionR2 :: Gen (Region R2)
genRegionR2 = arbitrary

genRegionR3 :: Gen (Region R3)
genRegionR3 = arbitrary

-- Testing

prop a = a === a where
    types = a::(Region R2)

propAnn (Annulus pnt r1 r2) = r1 < r2
    where types = pnt::R3
    
quickCheck $ forAll arbitraryAnnulusRegion propAnn
quickCheck prop

## Ploting with points and regions ##

In [ ]:
--generate genR2
pointlist = generate (listOf genR2')

plotR2Boundary = plotR2Lines . boundary
-- generate (boundary <$> arbitrarySquareRegion)

plotR2Boundarys r1 r2 = plotR2Boundary r1 >> plotR2Boundary r2

plotR2BoundaryAndPoints region points = plotR2Boundary region >> plotR2Points pl >> plotR2Points pl'
    where
    pl = filter (inDomain region) points
    pl' = filter (not . inDomain region) points

--ppp d = (plotR2 d) >> (plotR2' d)
--ppp' d b = (plotR2 d) >> (plotR2' b)

boundingBoxOfPoints points = plotR2Boundary region
    where
    (center, r) = findBoundingBox points
    region = Square center r

--toRenderable $ plotR2BoundaryAndPoints (Square (R2 0 0) 0.5) [R2 0 2, R2 1.5 1, R2 (-1) 0.5]
--toRenderable <$> (plotR2BoundaryAndPoints <$> (generate genRegionR2) <*> pointlist)
toRenderable <$> (plotR2BoundaryAndPoints <$> generate arbitrarySquareRegion <*> pointlist)

--toRenderable . plotR2Lines <$> generate (boundary <$> genRegionR2)
--toRenderable . plotR2Boundary <$> generate genRegionR2
--toRenderable <$> (plotR2Boundarys <$> generate genRegionR2 <*> generate genRegionR2)

--toRenderable . plotR2Boundary <$> generate (boundary <$> arbitraryDiskRegion)
--toRenderable . plotR2Boundary <$> generate (boundary <$> arbitraryAnnulusRegion)
--toRenderable <$> (plotR2 <$> pointlist)
--toRenderable <$> (plotR2' <$> pointlist)
--toRenderable <$> (ppp <$> pointlist)
--toRenderable <$> (ppp' <$> pointlist <*> pointlist)


## Plotting with curves and regions ##

In [ ]:
plotBezier contpoints = plotR2Line points >> boundingBoxOfPoints points
    where points = [bezierRedux t contpoints | t <- linspace 0 1 1000]
plotBezVerbose initpoints = plotBezier points >> plotR2Line points
    where points = take 5 initpoints

plotTestDeriv contpoints = plotCurve (bezierCurve 100 contpoints) >> plotCurve (bezierCurve' 100 contpoints)

--layout_y_axis . laxis_generate .= scaledAxis def (-10,80)

plotTestDeriv3 contpoints = do
    --layout_x_axis . laxis_generate .= scaledAxis def (-30,30)
    --layout_y_axis . laxis_generate .= scaledAxis def (-15,15)
    plotCurve points
    plotCurves lineSegmentsOrth
        where
        --lineSegments = [[(R2 px py), (R2 (px + 0.01*tx) (py + 0.01*ty))] | ((R2 px py), (R2 tx ty)) <- zip points tangents]
        lineSegmentsOrth = [[R2 px py, R2 (px + ty/sqrt (ty*ty + tx*tx)) (py - tx/sqrt (ty*ty + tx*tx))] | (R2 px py, R2 tx ty) <- zip points tangents]
        points = bezierCurve 100 contpoints
        tangents = bezierCurve' 100 contpoints

plotTestDeriv2 contpoints = plotCurves lineSegments >> plotCurves lineSegmentsOrth
    where
    lineSegments = [[R2 px py, R2 (px + 0.01*tx) (py + 0.01*ty)] | (R2 px py, R2 tx ty) <- zip points tangents]
    lineSegmentsOrth = [[R2 px py, R2 (px + 0.01*ty) (py - 0.01*tx)] | (R2 px py, R2 tx ty) <- zip points tangents]
    points = bezierCurve 100 contpoints
    tangents = bezierCurve' 100 contpoints

toRenderable . plotTestDeriv3 <$> pointlist
--toRenderable $ plotCurve (bezierCurve 100 [R2 0 0, R2 0.25 0.5, R2 0.75 0.5, R2 1 0])
--toRenderable $ plotCurve (bezierCurve' 100 [R2 0 0, R2 0.25 0.5, R2 0.75 0.5, R2 1 0])
--toRenderable . plotBezVerbose <$> pointlist

plotR2BoundaryAndCurve region curve = plotR2Boundary region >> plotCurves curvesIn >> plotCurves curvesOut
    where (curvesIn, curvesOut) = curveFilterBoundary region curve

toRenderable <$> (plotR2BoundaryAndCurve <$> generate genRegionR2 <*> (bezierCurve 1000 <$> pointlist))

--toRenderable . plotCurve . (bezierCurve 100) <$> pointlist
--toRenderable $ plotR2Line [bezierRedux t [a, b, R2 0.5 2, c] | t <- linspace 0 1 10]
--toRenderable $ plotR2Lines [[lerp t a b | t <- linspace 0 1 10], [lerp t b c | t <- linspace 0 1 10]]
--toRenderable $ plotR2Line [quad t a b c | t <- linspace 0 1 100]

--[lerp t a b | t <- linspace 0 1 10]

## Plotting Under Transformations ##

In [ ]:
testFunc :: R2 -> R2
testFunc (R2 a b) = R2 (a*10/(a*a + b*b)) (b*10/(b*b + a*a))

plotDomainAndRange func region = plotSideBySide (plotR2Boundary region >> plotR2Points [center]) (plotCurvesTransformed func (boundary region) >> plotR2Points [func center])
    where
    (center, r) = (findBoundingBox . head) $ boundary region

toRenderable . plotDomainAndRange testFunc <$> generate genRegionR2


plotR2BoundaryAndCurveDomainRange func region curve = plotSideBySide (plotCurves boundaryCurve >> plotCurves curvesIn >> plotCurves curvesOut >> plotR2Points [center])
                                                                     (plotCurvesTransformed func boundaryCurve >> plotCurvesTransformed func curvesIn >> plotCurvesTransformed func curvesOut >> plotR2Points [func center])
    where
    (curvesIn, curvesOut) = curveFilterBoundary region curve
    boundaryCurve = boundary region
    (center, r) = (findBoundingBox . head) boundaryCurve

--(a, b, c) = (R2 0 0, R2 2 2, R2 4 0)
--toRenderable $ plotR2BoundaryAndCurveDomainRange testFunc (Square (R2 0 0) 1) (bezierCurve 100 [a, b, c])
toRenderable <$> (plotR2BoundaryAndCurveDomainRange testFunc <$> generate genRegionR2 <*> (bezierCurve 1000 <$> pointlist))
toRenderable <$> (plotR2BoundaryAndCurveDomainRange testFunc <$> generate genRegionR2 <*> (bezierCurveClosed 1000 <$> pointlist))
--toRenderable <$> (plotR2BoundaryAndCurve <$> generate genRegionR2 <*> (bezierCurve 1000 <$> pointlist))

## Manifolds ##
validation of transforms

In [ ]:
instance Homeomorphism Chart where
    move chartA chartB = coordinateSystem chartB . parameterization chartA

foo1 :: R2 -> R3
foo1 (R2 a b) = R3 x y z
    where
     factor = a*a + b*b + 1
     (x, y, z) = (2*a/factor, 2*b/factor, (a*a + b*b - 1)/factor)

foo1Inv :: R3 -> R2
foo1Inv (R3 a b c) = R2 x y
    where
     lambda = 1 - c
     (x, y) = (a/lambda, b/lambda)

foo2 :: R2 -> R3
foo2 (R2 a b) = R3 x y z
    where
     factor = a*a + b*b + 1
     (x, y, z) = (2*a/factor, 2*b/factor, (1 - a*a - b*b)/factor)

foo2Inv :: R3 -> R2
foo2Inv (R3 a b c) = R2 x y
    where
     lambda = 1 + c
     (x, y) = (a/lambda, b/lambda)

-- for testing

propfb1 (R2 a b) = (foo1Inv . foo1) (R2 a b) === R2 a b
propfb2 (R2 a b) = (foo2Inv . foo2) (R2 a b) === R2 a b
propf1OnShell (R2 a b) = distanceSquared (foo1 (R2 a b)) (R3 0 0 0) ~= 1
propf2OnShell (R2 a b) = distanceSquared (foo2 (R2 a b)) (R3 0 0 0) ~= 1
propf1Ring theta = z ~= 0.0
    where
    point = R2 (cos theta) (sin theta)
    (R3 x y z) = foo1 point
propf2Ring theta = z ~= 0.0
    where
    point = R2 (cos theta) (sin theta)
    (R3 x y z) = foo2 point

--quickCheck propM
print "test foo1"
quickCheck propfb1
print "test foo2"
quickCheck propfb2
print "foo1 on shell"
quickCheck propf1OnShell
print "foo2 on shell"
quickCheck propf1OnShell
print "foo1 ring on plane"
quickCheck propf1Ring
print "foo2 ring on plane"
quickCheck propf2Ring

## Curves ##

In [ ]:
transformCurve :: (a -> b) -> Curve a -> Curve b
transformCurve function curve = [function point | point <- curve]

transformCurves :: (a -> b) -> [Curve a] -> [Curve b]
transformCurves function curves = [transformCurve function curve | curve <- curves]

moveCurve :: Chart a b -> Chart a b -> Curve a -> Curve a
moveCurve chartA chartB curve = transformCurve (move chartA chartB) curve

moveCurves :: Chart a b -> Chart a b -> [Curve a] -> [Curve a]
moveCurves chartA chartB curves = transformCurves (move chartA chartB) curves

projectCurveXY :: Curve R3 -> Curve R2
projectCurveXY curve = [R2 x y | (R3 x y z) <- curve]

projectCurveXZ :: Curve R3 -> Curve R2
projectCurveXZ curve = [R2 x z | (R3 x y z) <- curve]

projectCurveYZ :: Curve R3 -> Curve R2
projectCurveYZ curve = [R2 y z | (R3 x y z) <- curve]

derivativeCurve :: (LinearSpace a) => Curve a -> Curve a
derivativeCurve curve = [scalarMult (1/dt) delta | delta <- finiteDifference]
    where finiteDifference = [vectorAdd pointB (scalarMult (-1) pointA) | (pointA, pointB) <- zip (init curve) (tail curve)]
          parameterization = linspace 0 1 (length curve)
          dt = head [t_1 - t_0 | (t_0, t_1) <- zip (init parameterization) (tail parameterization)]

## Plotting on Manifold ##

In [ ]:
--map1 = Map foo1 foo1Inv
--map2 = Map foo2 foo2Inv
chart1 = Chart (Disk (R2 0 0) 10) foo1 foo1Inv
chart2 = Chart (Disk (R2 0 0) (1.0/5)) foo2 foo2Inv
atlas = [chart1, chart2]::(Atlas R2 R3)

prop point =
    not (distanceSquared point (R2 0 0) ~= 0.0) ==>
    move chart2 chart1 (move chart1 chart2 point) === point
quickCheck prop

plotManifoldChartDomains manifold = plotNSideBySide [plotCurves boundaryCurves | boundaryCurves <- boundaryCurvesRegions]
    where
    boundaryCurvesRegions = [boundary (domain chart) | chart <- manifold]

toRenderable $ plotManifoldChartDomains atlas

plotManifoldCharts manifold = plotNSideBySide [plotCurves boundaryCurves | boundaryCurves <- boundaryCurvesRegions]
    where
    boundaryCurvesRegions = [concat [ moveCurves chartA chartB (boundary (domain chartA)) | chartA <- manifold] | chartB <- manifold]

toRenderable $ plotManifoldCharts atlas


plotManifoldCurve manifold chartCurve = plotNSideBySide [plotCurves boundaryCurves >> plotCurve curveOnRegion | (boundaryCurves, curveOnRegion) <- zip boundaryCurvesRegions curveOnRegions]
    where
    (ChartCurve originalCurve originalChart) = chartCurve
    boundaryCurvesRegions = [concat [ moveCurves chartA chartB (boundary (domain chartA)) | chartA <- manifold] | chartB <- manifold]
    curveOnRegions = [moveCurve originalChart chartB originalCurve | chartB <- manifold]


testplot points =  plotManifoldCurve atlas (ChartCurve (bezierCurveClosed 1000 points) chart1)
toRenderable . testplot <$> pointlist

In [ ]:
plotCurveOnManifoldEmbedding manifold chartCurve = plotGrid [[plotCurves disk >> plotCurve (projectCurveXY curveOnManifold), 
                                                              plotCurves disk >> plotCurve (projectCurveXZ curveOnManifold)],
                                                             [plotCurves disk >> plotCurve (projectCurveYZ curveOnManifold),
                                                              plotCurves disk]]
    where
    (ChartCurve originalCurve originalChart) = chartCurve
    curveOnManifold = [parameterization originalChart point | point <- originalCurve]
    disk = boundary (Disk (R2 0 0) 1)

testplot2 points =  plotCurveOnManifoldEmbedding atlas (ChartCurve (bezierCurve 1000 points) chart1)
toRenderable . testplot2 <$> pointlist

## Continuity of Curves ##

In [ ]:
sort :: (Ord a) => [a] -> [a]
sort [] = []
sort (x:xs) = sort [y | y <- xs, y <= x] ++ [x] ++ sort [y | y <- xs, y > x]

getDistances curve = [R2 (i/fromIntegral (length curve)) (distanceSquared pointA pointB) | (i, pointA, pointB) <- zip3 [0 ..] (init curve) (tail curve)]
getDistances' curve = [distanceSquared pointA pointB | (i, pointA, pointB) <- zip3 [0 ..] (init curve) (tail curve)]

plotDistance2 points = plotGrid [[plotCurve (getDistances curve) >> plotCurve [R2 0 (10*cutoff), R2 1 (10*cutoff)] >> plotCurve [R2 0 median, R2 1 median] >> plotCurve [R2 0 large, R2 1 large], plotCurve curve]]
    where
    pntlength = length points
    curve = (bezierCurve 1000 (take (1 + div pntlength 2) points)) ++ (bezierCurve 1000 (drop (div pntlength 2) points))
--    curve = bezierCurve 1000 points
    distLtS = reverse $ sort (getDistances' curve)
    cutoff | length distLtS > 250 = head (drop 25 distLtS)
           | otherwise = head (drop (div (length distLtS) 2) distLtS)
    (median, large) = (head (drop (div (length distLtS) 2) distLtS), head distLtS)

isContinuous :: (MetricSpace a) => Curve a -> Bool
isContinuous curve = head sortedDistances < 10*cutoff
    where sortedDistances = reverse $ sort [distanceSquared pointA pointB | (pointA, pointB) <- zip (init curve) (tail curve)]
          numPoints = length curve
          cutoff = if (numPoints > 250)
                   then head (drop 25 sortedDistances)
                   else head (drop (div (length sortedDistances) 2) sortedDistances)
                   
toRenderable . plotDistance2 <$> pointlist

## Derivative of Curves ##

In [ ]:
plotCurveAndDerivative points = plotGrid [[plotCurve curve, (plotCurve . derivativeCurve) curve], [plotCurve [R2 t value | (t, value) <- zip (linspace 0 1 (length comp)) comp], plotCurve curve']]
    where pntlength = length points
          curve = bezierCurve 1000 points
          --curve' = bezierCurve' 1000 points
          curve' = [bezierRedux' t points | t <- (linavg 0 1 1000)]
          comp = [distanceSquared a b | (a, b) <- zip (derivativeCurve curve) curve']
          linavg a b num = [0.5*(a + b) | (a, b) <- zip (init space) (tail space)]
              where space = linspace a b num
          --curve = (bezierCurve 1000 (take (1 + div pntlength 2) points)) ++ (bezierCurve 1000 (drop (div pntlength 2) points))
          
toRenderable . plotCurveAndDerivative <$> pointlist

(isContinuous . derivativeCurve . bezierCurve 1000) <$> pointlist


## Differentiable Manifolds ##
a Differentiable manifold $M$ of dimension $n$ is a topological manifold with the following properties

Primary
1. The map between overlaping charts $f:\phi_\beta^{-1} \circ \phi_\alpha$ and its inverse $f^{-1}:\phi_\alpha^{-1} \circ \phi_\beta$ are differentiable
2. Each region of the manifold is covered by charts, so it can be constructed from the parameterizations $\bigcup_\alpha \phi_\alpha(U_\alpha) = M$

Secondary
- An atlas $\mathcal{A}=\{(U_\alpha, \phi_\alpha)\}$ must satisfy the two prevous statements. If every pair $(V \subset \mathbb{R}^n, \psi:V \to M)$, that satisfies statement (1.) relative to every pair $(U, \phi)$ in $\mathcal{A}$, is also in $\mathcal{A}$, then the atlas $\mathcal{A}$ is a **maximal atlas**

A differentiable manifold $M$ is a topological manifold of dimension $n$ with parameterizations and charts defined between the manifold and open subsets $U_\alpha \subset \mathbb{R}^n$ indexed by $\alpha$, parameterization $\phi_\alpha:U_\alpha \to M$, charts $\phi_\alpha^{-1}:M \to U_\alpha$ and they are inverses of each other $\phi_\alpha \circ \phi_\alpha^{-1} = \phi_\alpha^{-1} \circ \phi_\alpha = I$. Each region of the domain of $M$ must be covered by charts such that the union of all of the parametrizations is the manifold $\bigcup_\alpha \phi_\alpha(U_\alpha) = M$. Given two parametrizations map to regions of the manifold with an overlap $W$ then $\phi_\alpha(U_\alpha) \cap \phi_\alpha(U_\beta) = W \neq \emptyset$, and we can define a function $f:\phi_\alpha^{-1}(W) \to \phi_\beta^{-1}(W)$ as $f=\phi_\beta^{-1} \circ \phi_\alpha$ where $f$ and $f^{-1}=\phi_\alpha^{-1} \circ \phi_\beta$ are both differentiable.

In [ ]:
continuousChartGivenCurve :: (MetricSpace a) => Chart a b -> Chart a b -> Curve a -> Bool
continuousChartGivenCurve chartA chartB curve = isContinuous (moveCurve chartA chartB curve)

differentiableChartGivenCurve :: (LinearSpace a, MetricSpace a) => Chart a b -> Chart a b -> Curve a -> Bool
differentiableChartGivenCurve chartA chartB curve = isContinuous $ derivativeCurve (moveCurve chartA chartB curve)

propIsTopologicalManifold points =
    not (distanceSquared (head points) (R2 0 0) ~= 0.0) ==>
    continuousChartGivenCurve chart1 chart2 curve === True
        where curve = bezierCurve 1000 points
              
propIsDifferentiableManifold points =
    not (distanceSquared (head points) (R2 0 0) ~= 0.0) ==>
    differentiableChartGivenCurve chart1 chart2 curve === True
        where curve = bezierCurve 1000 points

quickCheck $ forAll (vectorOf 5 genR2) propIsTopologicalManifold
quickCheck $ forAll (vectorOf 5 genR2) propIsDifferentiableManifold

## Differentiable Maps ##

In [ ]:
foo3 :: R2 -> R3
foo3 (R2 a b) = R3 x y z
    where
     factor = (a*a + b*b + 1)/2
     (x, y, z) = (2*a/factor, 2*b/factor, (a*a + b*b - 1)/factor)

foo3Inv :: R3 -> R2
foo3Inv (R3 a b c) = R2 x y
    where
     lambda = 2*(1 - c/2)
     (x, y) = (a/lambda, b/lambda)
chart3 = Chart (Disk (R2 0 0) 10) foo3 foo3Inv

foo4 :: R2 -> R3
foo4 (R2 a b) = R3 x y z
    where
     factor = (a*a + b*b + 1)/2
     (x, y, z) = (2*a/factor, 2*b/factor, (1 - a*a - b*b)/factor)

foo4Inv :: R3 -> R2
foo4Inv (R3 a b c) = R2 x y
    where
     lambda = 2*(1 + c/2)
     (x, y) = (a/lambda, b/lambda)
chart4 = Chart (Disk (R2 0 0) (1.0/5)) foo4 foo4Inv
atlas2 = [chart3, chart4]::(Atlas R2 R3)

--function :: (LinearSpace a) => a -> a
--function point = point--scalarMult 1 point
function :: (LinearSpace a) => a -> a
function point = scalarMult 2 point

functionInv :: (LinearSpace a) => a -> a
functionInv point = scalarMult (1/2) point

-- If functionInv and function are continuous invertible bijections then they for a diffeomorphism between atlas and atlas2

localFunction :: Chart a b -> Chart c d -> (b -> d) -> (a -> c)
localFunction chartA chartB function = coordinateSystem chartB . function . parameterization chartA

continuousFunctionGivenCurve :: (MetricSpace c) => Chart a b -> Chart c d -> (b -> d) -> Curve a -> Bool
continuousFunctionGivenCurve chartA chartB function curve = isContinuous (transformCurve (localFunction chartA chartB function) curve)

differentiableFunctionGivenCurve :: (LinearSpace c, MetricSpace c) => Chart a b -> Chart c d -> (b -> d) -> Curve a -> Bool
differentiableFunctionGivenCurve chartA chartB function curve = isContinuous $ derivativeCurve (transformCurve (localFunction chartA chartB function) curve)

propIsContinuousFunction points =
    not (distanceSquared (head points) (R2 0 0) ~= 0.0) ==>
    continuousFunctionGivenCurve chart1 chart4 function curve === True
        where curve = bezierCurve 1000 points
              
propIsDifferentiableFunction points =
    not (distanceSquared (head points) (R2 0 0) ~= 0.0) ==>
    differentiableFunctionGivenCurve chart1 chart4 function curve === True
        where curve = bezierCurve 1000 points

quickCheck $ forAll (vectorOf 5 genR2) propIsContinuousFunction
quickCheck $ forAll (vectorOf 5 genR2) propIsDifferentiableFunction

## Tangent Vectors ##
Given a differentiable curve $c:(-\epsilon, \epsilon) \to M$ and the set of all smooth functions $C^\infty(p)$ defined as $f:M \to \mathbb{R}$ where $p=c(0)$. The **Tangent vector to the curve** at $p$ is the operator $\dot c(0):C^\infty \to \mathbb{R}$, defined as $\dot c(0)(f) = \frac{d (f \circ c)}{dt}(0)$. So composing the maps $c:(-\epsilon, \epsilon) \subset \mathbb{R} \to M$ and $f:M \to \mathbb{R}$, produces a function $f \circ c:\mathbb{R} \to \mathbb{R}$ which is clearly differentiable by normal means since it does not involve the manifold. Note given $c_1:(-\epsilon, \epsilon) \to \mathbb{R}$, $c_2:(-\epsilon, \epsilon) \to \mathbb{R}$, $\gamma:(-\epsilon, \epsilon) \to \mathbb{R}$ and $f:M \to \mathbb{R}$, where $c_1(0) = c_2(0) = \gamma(0) = p$, then $a\dot c_1(0)(f) + b\dot c_2(0)(f) = af'(c_1(0))\frac{dc}{dt} + bf'(c_2(0))\frac{d\gamma}{dt} = af'(p)\frac{dc_1}{dt} + bf'(p)\frac{dc_2}{dt} = f'(p)(a\frac{dc_1}{dt} + b\frac{dc_2}{dt}) = f'(p)\frac{d\gamma}{dt}=\dot \gamma(0)(f)$ iff $\frac{d\gamma}{dt} = a\frac{dc_1}{dt} + b\frac{dc_2}{dt}$. Also given $c:(-\epsilon, \epsilon) \to \mathbb{R}$, $f:M \to \mathbb{R}$ and $g:M \to \mathbb{R}$, where $c(0) = p$, then $\dot c(0)(af + bg) = (af + bg)'(p)\frac{dc}{dt} = af'(p)\frac{dc}{dt} + bg'(p)\frac{dc}{dt}=a\dot c(0)(f) + b\dot c(0)(g)$.

Like how a vector $v$ on a vector space $V$ can be defined as f

In [ ]:
tangentVectorFromCurve :: Curve a -> (a -> Double) -> Double
tangentVectorFromCurve curve function = head (drop (div (length c_dot) 2) c_dot)
    where c_dot = derivativeCurve (transformCurve function curve)

tangentVectorFromCurve [R2 (-1) (-1), R2 1 1] (\(R2 x y)-> x)
tangentVectorFromCurve [R2 (-1) (-1), R2 1 1] (\(R2 x y)-> y)

v = tangentVectorFromCurve [R2 (-1) (-1), R2 1 1]
addTangentVectors :: ((a -> Double) -> Double) -> ((a -> Double) -> Double) -> ((a -> Double) -> Double)
addTangentVectors vectorA vectorB function = (vectorA function) + (vectorB function)
--v2 = addTangentVectors v vwith
--v2 (\(R2 x y)-> y)
--v2 (\(R2 x y)-> x)

## Vectors ##

I keep getting hungup on what vectors are in differential geometry and how to think about them. In the following I will give my current thought on what vectors are and how they work.

**Vectors** are elements of a **Vector Space** that is a **Linear Space**. Trying to give vectors an intuitive geometric interpretation, like in other areas of physics, too early lead me to lots of confussion and contradictions and infact made the whole prossess less intuitive. So lets start from the begining.

A **Vector Space** , also called a **Linear Space**, on a field $F$ is a set $V$ along with vector addition, defined as $u+v:u \in V \times v \in V \to V$, and scalar multiplication , defined as $av:a \in F \times v \in V \to V$, the set $V$ along with vector addition must form an abelian group and satisfy the properties

Primary
- Distributivity: $a(u + v) = au + av$ and $(a + b)v = av + bv$

Nessisary to maintain sanity
- Compatibility: $a(bv) = (ab)v$
- Identity: $1v = v$ where $1$ is the multiplicative identity of $F$

Anything satisfying these conditions can rightfully be called a vector space, and elements of such a space are vectors. The problem with manifolds is that by design they are not linear (maps, charts, functions, etc deffining and defined on manifolds are ingeneral not linear) so we need some mechanism to make linear object from nonlinear ones. The natural candidate for this mechanism is differential calculus since the derivative is closly tied to linear aproximations. However the ordinary definition of the derivative and partial derivative require a linear space with a sence of distance to begin with $\frac{df}{dx}=\lim_{h \to 0} \frac{f(x + h) - f(x)}{h}$. So we need some way of seporating the differentiation that we are using to make things linear from the manifold. We can use functions and curves in a way simular to Encapsulation from computer science to safely contain the complexity of manifolds away from differentation. Given a $n$ dimensional smooth manifold $M$ a curve $c:(-\epsilon, \epsilon) \in \mathbb{R} \to M$ and a function $f:M \to \mathbb{R}$, then the composition of the function and the curve is $f \circ c:\mathbb{R} \to \mathbb{R}$ which is a differentiable function from real numbers to real numbers and so can be differentiated by normal means with out any issues. After composing a sutible function with a sutible curve all of the subtleties and complexity of the manifold is hidden within the composite function which presents itself as an ordinary function of one real variable and as such we can apply the definition of the derivative. Now the actual object of intrest which we will call a "vector" is the oporator $v:C^\infty(M) \to \mathbb{R}$ which when acting on a smooth function $f$, on the manifold $M$, is defined as $v(f) = \left. \frac{d}{dt} f(c(t)) \right|_{t=0}$ where $c$ is some smooth curve on the manifold $M$ such that its behavour in the neborhood of the point $p=c(0)$ defines the vector $v$. Now is it valid to call this operator which takes smooth functions to real numbers a vector. Note that given two smooth functions on the manifold $f$ and $g$ then $v(f + g) = \left. \frac{d}{dt} (f(c(t)) + g(c(t))) \right|_{t=0} = \left. \frac{d}{dt} f(c(t)) \right|_{t=0} + \left. \frac{d}{dt} g(c(t)) \right|_{t=0} = v(f) + v(g)$, and that $v(af) = \left. \frac{d}{dt} (a \cdot f(c(t))) \right|_{t=0} = \left. a\frac{d}{dt} f(c(t)) \right|_{t=0} = a \cdot v(f)$. And finaly that given any two curves $a(t)$ and $b(t)$ on the manifold $M$ the sum of the resulting vector operators $v$ defined as $v(f) = \left. \frac{d}{dt} f(a(t)) \right|_{t=0} + \left. \frac{d}{dt} f(b(t)) \right|_{t=0}$ and there always exists some curve $c$ such that $\left. \frac{d}{dt} f(c(t)) \right|_{t=0} = \left. \frac{d}{dt} f(a(t)) \right|_{t=0} + \left. \frac{d}{dt} f(b(t)) \right|_{t=0}$ for every smooth function $f:M \to \mathbb{R}$. So the set of operators defined from the set of smooth curves on the manifold $M$ with $c(0)=p \in M$ forms a linear space that is it forms a vector space $T_pM$ and the individual elements of the space are vectors. Now if for each coordinate chart of $M$ we defined a set of $n$ functions $f_k:M \to \mathbb{R}$ subject to some constraints , they must form a valid alternative parameterization of the manifold on that chart, taking a vector $v$ and acting it on each of the functions $f_k$ yields a set of $n$ real numbers $v(f_k)$ which can be combined into an equivelent vector expressed as a linear combination of basis vector $v(g) = \sum_{k=0}^n v(f_k) \frac{\partial g}{\partial f_k}$ and this alowes us to view the operator $v$ as an arrow on the manifold constructed from the sum of arrows pointing along the direction of greatest increas of $f_k$.

## Tangent Vectors take 2 ##
define tangent vectors of a manifold $M$ at a point $p \in M$ as operators $v:C^\infty(M) \to \mathbb{R}$ as described in the previuos section

In [ ]:
data Vector a = TangentVector a ((a -> Double) -> Double) | ChartVector ((a -> Double) -> Double) | ZeroVector

getCenterOfList :: [a] -> a
getCenterOfList list = list !! max 0 halfLength
    where halfLength = div (length list) 2

instance (Eq a) => LinearSpace (Vector a) where
    vectorAdd (TangentVector q u) (TangentVector p v) | p == q = TangentVector p (\function -> u function + v function)
                                                      | otherwise = error "Undefined"
    vectorAdd (ChartVector u) (ChartVector v) = ChartVector (\function -> u function + v function)
    vectorAdd ZeroVector v = v
    vectorAdd v ZeroVector = v
    scalarMult a (TangentVector p v) = TangentVector p (\function -> a * v function)
    scalarMult a (ChartVector v) = ChartVector (\function -> a * v function)
    scalarMult a ZeroVector = ZeroVector
    vectorZero = ZeroVector --ChartVector (const 0.0)
--    vectorZero = error "Undefined" -- TangentVector (const 0.0)

tangentVectorFromCurve :: Curve a -> Vector a
tangentVectorFromCurve curve = TangentVector (getCenterOfList curve) (vectorOnFunctionFromCurve curve)

vectorOnFunctionFromCurve :: Curve a -> (a -> Double) -> Double
vectorOnFunctionFromCurve curve function = getCenterOfList c_dot
   where c_dot = derivativeCurve (transformCurve function curve)

chartVectorFromCurve :: Curve a -> Vector a
chartVectorFromCurve curve = ChartVector (vectorOnFunctionFromCurve curve)

chartVectorFromBasisR2 :: [Double] -> Vector R2
chartVectorFromBasisR2 components = foldl vectorAdd vectorZero [scalarMult component (chartVectorFromCurve basisCurve) | (component, basisCurve) <- zip components curveBasisR2]

chartVectorFromBasisR3 :: [Double] -> Vector R3
chartVectorFromBasisR3 components = foldl vectorAdd vectorZero [scalarMult component (chartVectorFromCurve basisCurve) | (component, basisCurve) <- zip components curveBasisR3]

tangentVectorFromBasis :: (Eq b) => Chart R2 b -> b -> [Double] -> Vector b
tangentVectorFromBasis chart point components = foldl vectorAdd vectorZero [scalarMult component basisVector | (component, basisVector) <- zip components (tangentBasisVectors chart point)]

tangentBasisVectors :: Chart R2 b -> b -> [Vector b]
tangentBasisVectors chart p = [tangentVectorFromCurve curve | curve <- curvesThroughP]
    where
        pChart = coordinateSystem chart p
        curvesThroughPChart = [[vectorAdd pChart point | point <- curve] | curve <- curveBasisR2]
        curvesThroughP = [transformCurve (parameterization chart) curve | curve <- curvesThroughPChart]

curveBasisR2 = [[lerp t (scalarMult (-1) point) point | t <- linspace 0.25 0.75 101] | point <- [R2 1 0, R2 0 1]]
curveBasisR3 = [[lerp t (scalarMult (-1) point) point | t <- linspace 0.25 0.75 101] | point <- [R3 1 0 0, R3 0 1 0, R3 0 0 1]]

plotChartVector curve = plotCurve curve >> plotCurve arrow
    where
        arrowLength = 0.1
        (ChartVector vector) = chartVectorFromCurve curve
        componentX = arrowLength * vector (\(R2 x y)->x)
        componentY = arrowLength * vector (\(R2 x y)->y)
        root = getCenterOfList curve
        arrow = [vectorAdd root (R2 (-componentX) (-componentY)), root, vectorAdd root (R2 componentX componentY)]

plotTangentVector chart curve = plotGrid [[plotCurves disk >> plotCurve (projectCurveXY curveM) >> plotCurve (projectCurveXY arrow), 
                                                              plotCurves disk >> plotCurve (projectCurveXZ curveM) >> plotCurve (projectCurveXZ arrow)],
                                                             [plotCurves disk >> plotCurve (projectCurveYZ curveM) >> plotCurve (projectCurveYZ arrow),
                                                              plotCurve curve]]
    where
        arrowLength = 0.3
        curveM = transformCurve (parameterization chart) curve
        (TangentVector point vector) = tangentVectorFromCurve curveM
        componentX = arrowLength * vector (\(R3 x y z)->x)
        componentY = arrowLength * vector (\(R3 x y z)->y)
        componentZ = arrowLength * vector (\(R3 x y z)->z)
        root = point
        arrow = [vectorAdd root (R3 (-componentX) (-componentY) (-componentZ)), root, vectorAdd root (R3 componentX componentY componentZ)]
        disk = boundary (Disk (R2 0 0) 1)


--                                plotCurve (projectCurveXY curveM) >> plotCurve (projectCurveXY arrow)
toRenderable . plotChartVector <$> (bezierCurve 1000 <$> pointlist)
toRenderable . plotTangentVector chart1 <$> (bezierCurve 1000 <$> pointlist)

(ChartVector v) = chartVectorFromBasisR2 [2, 2]
--(TangentVector p v) = tangentVectorFromCurve [R2 (-1) (-1), R2 1 1]
v (\(R2 x y)-> x)
v (\(R2 x y)-> y)

(TangentVector p v) = tangentVectorFromBasis chart1 (parameterization chart1 (R2 1 0)) [2, 2]
p
v (\(R3 x y z) -> x)
v (\(R3 x y z) -> y)
v (\(R3 x y z) -> z)


## Tangent Bundle and differental ##
The **tangent bundle** $TM$ is defined as $TM = \bigcup_{p \in M} T_pM$ and is a $2n$ dimensional smooth manifold. For each parameterization $\phi_\alpha:U_\alpha \subset \mathbb{R}^n \to M$ there exists a parameterization $\psi_\alpha:U_\alpha \times \mathbb{R}^n \to TM$. Given $\phi_\alpha(x^1, x^2, \dots x^n)$ the $\psi_\alpha$ can be defined as $\psi_\alpha(x^1, x^2, \dots x^n, v^1, v^2, \dots v^n) = \sum_{k=1}^n v^k \left( \frac{\partial}{\partial x^k} \right)_{\phi_\alpha(x^1, x^2, \dots x^n)}$ and the set $\{(U_\alpha \times \mathbb{R}^n, \psi_\alpha)\}$ is an atlas for the tangent bundle $TM$.

The **push-forward** of a differentiable map $f:M \to N$ is the map $df:TM \to TN$, it is denoted as $f_*$ and is a differentiable map between the smooth manifolds $TM$ and $TN$ that is the tangent bundles of $M$ and $N$.

In [ ]:
df :: (a -> b) -> Vector a -> Vector b
df function (TangentVector point vector) = TangentVector (function point) (\g -> vector (g . function))

## Ideas for version 2 ##
Make a system for defining topology, for example overlaping rectangles. A curve would need to be defined localy on each chart.

Use Automatic Differentiation to define any object that must be differentiable.

Combine curves with their parameterization ```type Curve a = [(a, Double)] -- [(point,t)] = (x(t), y(t), z(t))```

Have difernt types to distinguish points on the manifold and points on charts. ex
```Haskell
data Rn = R1 Double | R2 Double Double | R3 Double Double Double | ...
data Manifold a = Manifold a
function :: Manifold R3 -> R1
function (Manifold (R3 x y z)) = ...
```
One of the possible types for points must be vectors so we can define the tangent bundle manifold

In [ ]:
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Backend.Cairo

import Data.Time.LocalTime
import System.Random

values :: [ (Double,Double,Int,Int) ]
values = [ (d, v, z, t) | ((d,v,_),z,t) <- zip3 [(0,1,1), (1,2,3), ((-1), 0, 10)] zs ts ]
  where
    zs     = randoms $ mkStdGen 0
    ts     = randomRs (-2,27) $ mkStdGen 1

toRenderable $ do
  layout_title .= "Price History"
  layout_background .= solidFillStyle (opaque white)
  layout_foreground .= (opaque black)
  layout_left_axis_visibility . axis_show_ticks .= False

  --plot (line "price 1" [ [ (d,v) | (d,v,_) <- prices1] ] )

  plot $ liftEC $ do
    area_spots_4d_title .= "random value"
    area_spots_4d_max_radius .= 20
    area_spots_4d_values .= values